In [ ]:
from camera import Camera
from dorna2 import Dorna
from dorna_vision import Detection
import config
import time

# robot
robot = Dorna()
robot.connect(config.robot_ip)

# camera
camera = Camera()
camera.connect()

# detection
detection_cardamom = Detection(camera=camera, robot=robot, **config.detection_prm["cardamom"])
detection_digit = Detection(camera=camera, robot=robot, **config.detection_prm["digit"])

"""
init motion
"""
robot.set_alarm(0) # clear alarm
robot.play_list(cmd_list=config.work_init)


"""
main loop
"""
print("main loop")
robot.play_list(cmd_list=config.main_loop)
start_time = time.time()
tare_counter = config.tare_cycle # number of time an item has been weighted
stat = {"try": 0, "picked": 0, "failed":0,  "good":0, "bad":0, "time_passed":0, "avg_time":0} # stat
consecutive_failiure_counter = 0
consecutive_mix = 0

for k in range(config.cycles):

    # update time
    stat["time_passed"] = time.time()-start_time
    stat["avg_time"] = stat["time_passed"] /max(1,stat["picked"])
    
    """
    failed counter
    """
    if consecutive_failiure_counter > config.bin_image_thr and consecutive_mix < config.mix_thr:
        consecutive_failiure_counter = 0 # restart
        robot.play_list(cmd_list=config.bin_mix)
        consecutive_mix += 1 # update consecutive mix
        continue

    """
    tare cycle
    """
    if tare_counter >= config.tare_cycle:
        tare_counter = 0
        robot.play_list(cmd_list=config.tare)
        continue
    
    """
    search for candidate
    """
    robot.play_list(cmd_list=config.bin_image) # bin region
    cardamoms = detection_cardamom.run()
            
    #sol = False
    if not cardamoms: # no candidate
        consecutive_failiure_counter += 1
        continue
    consecutive_failiure_counter = 0 # reset counter
    consecutive_mix = 0 # reset counter

    
    """
    picking
    """
    config.pick[0] = {**config.pick[0], **{"x": cardamoms[0]["xyz"][0], "y": cardamoms[0]["xyz"][1], "z": cardamoms[0]["xyz"][2]+config.tcp_length+8}} # adjust pick xyz
    robot.play_list(cmd_list=config.pick) # pick
    stat["try"] += 1 # update stat

    """
    bad after pick
    """
    if cardamoms[0]["cls"] == "bad": # bad
        robot.play_list(cmd_list=config.bad_bin_after_pick) # drop
        stat["bad"] += 1 # update stat
        continue
    
    """
    go up
    """
    robot.play_list(cmd_list=[config.bin_image[0]])            
    
    """
    weight
    """
    cls = 0
    robot.play_list(cmd_list=config.weight)
    try:
        if float(detection_digit.run()[0]["cls"]) >= config.weight_thr:
            cls = 1
    except:
        pass
    

    """
    sort
    """
    if cls <= 0: # bad
        robot.play_list(cmd_list=config.drop_bad)
        stat["bad"] += 1 #update stat
    else: # good
        robot.play_list(cmd_list=config.drop_good[0:4])
        stat["good"] += 1 #update stat
    
    # increase weight counter
    stat["picked"] += 1 #update stat
    tare_counter += 1 # update stat


print("end")
